In [1]:
import copy
import numpy as np
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.utilities.risk_reporting import (createValueReport)

In [2]:
IndexManager.instance()

In [ ]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [5]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]

In [6]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

['SOFR-1B']
Curve components: dict_keys(['SOFR-1B'])


In [7]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    strike=95.7,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

a = ve.value[1]
print("Baseline PV (a):", a)


RFR Future PV: ['USD', np.float64(-2.811610301073455e-08)]
Baseline PV (a): -2.811610301073455e-08


In [8]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([99999999.99999999,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [9]:
yc.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,  -98.924743  ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,  -97.89682633,    0.        ,
           0.        ,    0.        , 

In [10]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, rfr_future, request="all", space="pv")

In [11]:
report['param_risk']

array([-1000000.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.,       -0.,       -0.,       -0.,       -0.,
             -0.,       -0.])

In [12]:
report = createValueReport(vp, yc, rfr_future, request="all", space="quote")

In [13]:
report['quote_risk']

array([989247.43001761,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
            0.        ,      0.        ,      0.        ,      0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ,     -0.        ,     -0.        ,
           -0.        ,     -0.        ])

V = df(0, T + 3M) (Fut - K)

dV/dFut = df 

dV/ddf = (Fut - K)

risk a disc * (Fut - K)

In [14]:
95.90 - 95.7

0.20000000000000284

In [15]:
yc.discountFactorGradientWrtModelParameters(index="SOFR-1B",to_date="2026-06-24")

In [16]:
yc.gradient_

array([-0.2422043, -0.2422043, -0.2422043, -0.       , -0.       ,
       -0.       , -0.       , -0.       , -0.       , -0.       ,
       -0.       , -0.       , -0.       , -0.       , -0.       ,
       -0.       , -0.       , -0.       , -0.       , -0.       ,
       -0.       , -0.       , -0.       , -0.       , -0.       ,
       -0.       ])

In [17]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.71],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [18]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [19]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [20]:
yc2 = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

['SOFR-1B']
Curve components: dict_keys(['SOFR-1B'])


In [21]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    strike=95.7,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc2,
    {"FUNDING INDEX": "SOFR-1B"},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

b = ve.value[1]

RFR Future PV: ['USD', np.float64(-9892.721677391173)]


In [22]:
a

np.float64(-2.811610301073455e-08)

In [23]:
b

np.float64(-9892.721677391173)

In [24]:
(a-b)/0.01

np.float64(989272.1677363057)

In [25]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2029-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.0369,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(367.0503073610889)] Par Rate: 0.03680000000000002


In [26]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([-251480.65146573, -251480.65146573, -251014.98403621,
       -251014.98403621, -242484.78341184, -242484.78341184,
       -242039.83397935, -242039.83397935, -235121.07323957,
       -235121.07323957, -233384.3228237 , -233384.3228237 ,
       -901261.27830474,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [27]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, ois_swap, request="all", space="quote")

In [28]:
report['param_risk']

array([ 7.35765789e-01,  7.43763155e-01,  6.26907736e-01,  6.33476336e-01,
        5.21377485e-01,  5.26543757e-01,  4.06931757e-01,  4.10841919e-01,
        2.95119675e-01,  2.97820184e-01,  1.76877310e-01,  1.78459379e-01,
        1.00007234e+06, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [29]:
report['quote_risk']

array([-7.27854416e-01, -7.28120524e-01, -6.07359005e-01, -6.07512648e-01,
       -4.95102372e-01, -4.95253849e-01, -3.79107005e-01, -3.79183789e-01,
       -2.69908905e-01, -2.69949477e-01, -1.58903416e-01, -1.58923543e-01,
        3.67076861e+06, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00])

In [30]:
swap = ProductOvernightSwap(
    effectiveDate="2026-09-24",
    maturityDate="2027-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.0369,
    notional=1000000,
    position="LONG"
)

In [31]:
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

a= ve.value[1]

OIS Swap PV: ['USD', np.float64(-1688.764478070938)] Par Rate: 0.038712447531724814


In [32]:
ve.calculateFirstOrderRisk()
ve.firstOrderRisk

array([    422.19111952,     422.19111952,     422.19111952,
           422.19111952, -240663.26221304, -240663.26221304,
       -240218.31278055, -240218.31278055,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [33]:
vp = {"FUNDING INDEX": "SOFR-1B"}
report = createValueReport(vp, yc, swap, request="all", space="pv")

In [34]:
report['param_risk']

array([  -4.2219112 ,   -4.26780102,   -4.31261294,   -4.35779954,
       2509.48654296, 2534.35276628, 2553.95194153, 2578.49258196,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ,   -0.        ,   -0.        ,
         -0.        ,   -0.        ])

In [35]:
report = createValueReport(vp, yc, swap, request="all", space="quote")
report['quote_risk']

array([    4.1765148 ,     4.17804176,     4.17813364,     4.17919058,
       -2383.01954623, -2383.74863523, -2379.32050148, -2379.80240827,
           0.        ,     0.        ,     0.        ,     0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ,    -0.        ,    -0.        ,
          -0.        ,    -0.        ])

In [ ]:
MARKET_DF_3 = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.09],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [37]:
data_objs_3, dc_3 = build_yc_data_collection(MARKET_DF_3)

In [38]:
build_methods_3 = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [39]:
yc3 = YieldCurve("2025-09-24", dc, build_methods_3)
print("Curve components:", yc3.components.keys())

['SOFR-1B']
Curve components: dict_keys(['SOFR-1B'])


In [40]:
ve = ValuationEngineRegistry().new_valuation_engine(
    yc3, {"FUNDING INDEX": "SOFR-1B"}, swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(-1688.806708331933)] Par Rate: 0.03871244753172459


In [ ]:
b = ve.value[1]

In [42]:
a

np.float64(-1688.764478070938)

In [43]:
b

np.float64(-1688.806708331933)

In [44]:
(a-b)/0.01

np.float64(4.223026099498384)

In [45]:
### you have a function 

def risk_calc(model, vp, product):

    ve = "ve" # <= valuation engine initialized 
    ve.calculateValue()
    # initialize a gradient
    gradient = np.array()
    ve.calculateFirstOrderRisk(gradient, accumulate=False, scaler=1)
    ### inside of ve engine
    ### gradient.resize(model.numParameters())
    ### mode.discountFactorGradientWrtModelParameters(t1, gradient=gradient, scaler=dVdDF(t1))
    ### mode.discountFactorGradientWrtModelParameters(t2, gradient=gradient)
    ### mode.discountFactorGradientWrtModelParameters(t3, gradient=gradient)
    
    
    ### 
    model.postprocess(..., gradient)
    ### inside postprocess
    ### fetch my jacobin
    ### multiply through your gradient
    ### and then format the risk report

